In [1]:
from sklearn.tree import DecisionTreeClassifier

In [2]:
X = [[0,0],[1,1]]
Y = [0,1]

clf = DecisionTreeClassifier()
clf.fit(X, Y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [3]:
clf.predict([[2,2]])

array([1])

In [24]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split, cross_val_score
import pydotplus
import pandas as pd

In [5]:
iris = load_iris()

clf = DecisionTreeClassifier()
clf.fit(iris.data, iris.target)
dot_data = export_graphviz(clf, out_file=None)
graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_pdf('./数据集/iris.pdf')

True

使用决策树来检测POP3暴力破解，仍然使用KDD99的数据集，提取相关的网络特征及TCP协议内容相关特征。

In [6]:
def load_and_get_features_label(filename, train=True, test=False):
    if train:
        kdd99_train = pd.read_csv(filename, header=None)
        pop3_train = kdd99_train[kdd99_train[41].str.contains('normal|guess_passwd', regex=True)]
        pop3_train = pop3_train[pop3_train[2] == 'pop_3']
        features_train = pd.concat([pop3_train[0],pop3_train[pop3_train.columns[4:8]],pop3_train[pop3_train.columns[22:30]]], axis=1)
        label = pop3_train[pop3_train.columns[-1]]
        return features_train,label
    if test:
        kdd99_test = pd.read_csv(filename, header=None)
        features_test = pd.concat([kdd99_test[0],kdd99_test[kdd99_test.columns[4:8]],kdd99_test[kdd99_test.columns[22:30]]], axis=1)
        return features_test

In [7]:
features_train, label = load_and_get_features_label('./数据集/KDD 99/kddcup.data.corrected')

In [8]:
clf = DecisionTreeClassifier()
X_train, X_val, y_train, y_val = train_test_split(features_train, label)
clf.fit(X_train, y_train)
clf.score(X_val, y_val)

1.0

In [9]:
features_test = load_and_get_features_label('./数据集/KDD 99/kddcup.testdata.unlabeled', train=False, test=True)

In [10]:
X_test = features_test
y_predict = clf.predict(X_test)

In [15]:
y_predict[y_predict == 'guess_passwd']

array([], dtype=object)

In [16]:
label.unique()

array(['normal.'], dtype=object)

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
import glob
import numpy as np

In [22]:
def features_and_label(path, train=True, attack=False):
    if train:
        fileList = glob.glob(path + '*.txt')
        featuresList_train = []
        for file in fileList:
            featuresList_train.append(' '.join(np.loadtxt(file).astype('int').astype('str')))
        label_train = [0] * len(fileList)
        return featuresList_train, label_train
    if attack:
        fileList = glob.glob(path + '*.txt')
        featuresList_attack = []
        for file in fileList:
            featuresList_attack.append(' '.join(np.loadtxt(file).astype('int').astype('str'))) 
        label_attack = [1] * len(fileList)
        return featuresList_attack, label_attack

In [36]:
normal, normal_label = features_and_label('./数据集/ADFA-LD/Training_Data_Master/')
attack, attack_label = features_and_label('./数据集/ADFA-LD/Attack_Data_Master/Hydra_FTP_*/', train=False, attack=True)

countvec = CountVectorizer()
X = countvec.fit_transform(normal + attack)
y = normal_label + attack_label

X_train, X_val, y_train, y_val = train_test_split(X, y)

clf = DecisionTreeClassifier()
rfc = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
rfc.fit(X_train, y_train)
clf.score(X_val, y_val)

0.9678714859437751

In [37]:
cross_val_score(clf, X, y, cv=10).mean()

0.9657270727072709

In [38]:
cross_val_score(rfc, X, y, cv=10).mean()

0.9889088908890888

In [27]:
dot_data = export_graphviz(clf)
graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_pdf('./数据集/ftp.pdf')

True

In [28]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

In [29]:
X, y = make_blobs(n_samples=10000, n_features=10, centers=100, random_state=0)

In [32]:
clf = DecisionTreeClassifier(max_depth=None, min_samples_split=2, random_state=0)
cross_val_score(clf, X, y, cv=10).mean()

0.9828000000000001